In [1]:
import numpy as np
import pandas as pd
import re

pd.options.display.float_format = '{:.2f}'.format # change the display format of numbers

In [2]:
saffronart_df = pd.read_csv('saffronart_dataset.csv') 

# change the dates to Pandas datetime objects:
saffronart_df['auction_date'] = pd.to_datetime(saffronart_df['auction_date'])
saffronart_df['birth_date'] = pd.to_datetime(saffronart_df['birth_date'])
saffronart_df['death_date'] = pd.to_datetime(saffronart_df['death_date'])

saffronart_df.head(5)

,artist,birth_date,birth_place,death_date,artist_age,title,winning_bid,low_est,high_est,auction_date,category,style,size,medium
0,akhilesh,1956-01-01,Indore,NaT,63.00,Divine Conversation,9000.00,5000.00,7000.00,2019-06-01,painting,abstract,71.5 x 71.5 in,acrylic on canvas
1,akhilesh,1956-01-01,Indore,NaT,63.00,In search of Untitled forms -II,1120.00,5000.00,6670.00,2013-11-01,painting,abstract,32.5 x 44 in,acrylic on canvas
2,akhilesh,1956-01-01,Indore,NaT,63.00,In Search of Untitled Lines,6186.00,6900.00,8625.00,2013-08-01,painting,NaN,47 x 47 in,acrylic on canvas
3,akhilesh,1956-01-01,Indore,NaT,63.00,Magadhi,2400.00,5770.00,7695.00,2013-02-01,painting,abstract,33 x 44.5 in,acrylic on canvas
4,akhilesh,1956-01-01,Indore,NaT,63.00,Untitled,2942.00,6735.00,8655.00,2013-02-01,painting,abstract,40 x 40 in,acrylic on canvas


---
# Compute Area and Volume
## Detect issues and flawed records:

In [3]:
# play with regex:

# if there're more than one piece, they're often marked with alphabet chars:
abc_order = '.*[a-z]\)' # ex: a), b), c), ... z)

# they can also mention the number of pieces in the lot or mention their postions (top/bottom)
pieces_num = ['.*set of', ' two ', ' three ', ' four ', ' five ', ' six ', 
              ' seven ', ' eight ', ' nine ', ' ten ', ' top ', ' bottom |']

multi_pi_ind = '|.*'.join(pieces_num) # multiple pieces indicator
multi_pi_ind += abc_order
print(multi_pi_ind)

s1 = 'height a): 14 x 11.5 in '
s2 = 'this work comprises of two parts'
s3 = 'c) and e) 11 x 7 in'

for i, s in enumerate([s1, s2, s3]):
    print('s{}:'.format(i+1), bool(re.match(multi_pi_ind, s)))
    
bool(re.match('.*(\d+|\d+\.\d+)\sx\s(\d+|\d+\.\d+)', 'height : 14 x 11.5 in')) 

.*set of|.* two |.* three |.* four |.* five |.* six |.* seven |.* eight |.* nine |.* ten |.* top |.* bottom |.*[a-z]\)
s1: True
s2: True
s3: True


True

In [4]:
def check_size(row):
    dimensions = re.findall('\d+\.\d+|\d+', row['size'])
    dimensions = list(map(float, dimensions)) # turn all dimensions into float type
    
    if not all(d > 0 for d in dimensions): # check if there's any dimensions not greater than 0
        print(row.name, row['size'], row['medium'], sep=' | ')
    
    if len(dimensions)<2 or len(dimensions)>3: #cannot have more than 2 or 3 dimensions
        print(row.name, row['size'], row['medium'], sep=' | ')
    
    if bool(re.match(multi_pi_ind, row['size'])):
        print(row.name, row['size'], row['medium'], sep=' | ')
           
saffronart_df.apply(check_size, axis=1)
print()

55 |  a) height : 14 x 11.5 in |  fiberglass and paint 
1254 |  b) 15 x 12.5 in |  a) 12 x 9.5 in (30.4 x 24.1 cm) 
2593 |  two panels measuring 13.5 x 8.5 in |  laminated photo-copies and fabric transfer mounted on board 
3041 |  10.5 x 0 x 3 in |  bronze 
7651 |   c) 3.5 x 5.5 in |  a) b) 5.5 x 3.5  in (14 x 8.8 cm)  
7669 |  published: exhibited: | â â 146 cm x 105.4 cm  
7714 |  this work comprises of two parts, a) measuring 7.5 x 7.25 x 15.5 inches |  depth: 15.5 in (39.4 cm) 
7961 |  c) and e) 11 x 7 in |  a), b), d) and f) 11 x 7.5 in (28 x 19 cm) 
8466 |  b) 4.5 x 4.5 in |  a) 10.5 x 8.5 in (26.6 x 21.5 cms) 
8515 |  b) 11 x 8.5 in |  a) & c) 10 x 8 in (25.4 x 20.3 cm) (each) 
8531 |  c) 8 x 7.5 in |  b) 8 x 7 in (20 x 18 cm) 
8617 |  9 in x 0 in | bronze 
9844 |  b) 17.5 x 17.5 in |  a) 17 x 17 in (43.1 x 43.1 cm) 
10122 |  24 x 0 in |  bronze 
10123 |  20 x 0 x 3 in | teracotta 
10124 |  13.5 x 0 in |  bronze 
10598 |  c) 24 x 37.5 in |  b) 30 x 11 in (76.2 x 27.9 cm) 
11841 

In [5]:
# play with regex: 

avoid_ex_list = ['^(\d+|\d+\.\d+)\sx\s(\d+|\d+\.\d+)\sin\.?$',            # ex: 9 x 3 in, 9.5 x 3 in
                 '^(\d+|\d+\.\d+)\sx\s(\d+|\d+\.\d+)\sin\.?\s\(each\)$',  # ex: 9 x 3 in (each)
                 '^(\d+|\d+\.\d+)\sin\sx\s(\d+|\d+\.\d+)\sin\sâ\sâ$']     # ex: 9 in x 3 in â â

avoid_ex = '|'.join(avoid_ex_list)

s1 = '14 x 10.5 in (each)'
s2 = '38 in x 43 in â â'
s3 = 'b) 11.5 x 9 in'
s4 = '75.5 x 22.5 in'
s5 = 'image size: 7.5 x 9.5 in'

for i, s in enumerate([s1, s2, s3, s4, s5]):
    print('s{}:'.format(i+1), bool(re.match(avoid_ex, s)))

s1: True
s2: True
s3: False
s4: True
s5: False


In [6]:
def check_medium(row):
    if bool(re.match(avoid_ex, row['medium'].strip())): # check if it matches the specified expression
        return                                          # if matched then pass
    
    if any(char.isdigit() for char in row['medium']) or bool(re.match(multi_pi_ind, row['medium'])): 
    # check if there's any number in medium or if it indicates multiple pieces
        print(row.name, row['size'], row['medium'], sep=' | ')
        
saffronart_df.apply(check_medium, axis=1)
print()

302 |  29.5 x 21.5 in |  270gsm, gold foil on gf smith paper and mounted veneer 
307 |  37 x 25.5 in | painted lithograph on 250 gsm velin d arches paper pasted on board 
308 |  37 x 25.5 in | painted lithograph on 250 gsm velin d arches paper pasted on board 
483 |  24 x 36 in |  ultra chrome k3 pigments on hahnemuhle photo acid free paper 
484 |  24 x 36 in |  ultra chrome k3 pigments on hahnemuhle photo acid free paper 
485 |  24 x 35.5 in |  ultra chrome k3 pigments on hahnemuhle photo acid free paper 
486 |  24.5 x 35.5 in |  ultra chrome k3 pigments on hahnemuhle photo acid free paper 
990 |  29.2 x 22.8 cm |  b) 11.5 x 9 in 
1254 |  b) 15 x 12.5 in |  a) 12 x 9.5 in (30.4 x 24.1 cm) 
1378 |  5 x 3.5 in |  b) ink on paper pasted on mountboard 
1439 |  10.5 x 14.5 in |  b) mixed media on handmade paper 
1552 |  44.25 x 30 in |  this work comprises of four panels 
1661 |  13.75 in x 34 in |  digital prints on duratrans/ clear films, 3 layers 
1662 |  13.75 in x 34 in |  digital pri

In [7]:
multi_pieces = [55, 990, 1254, 1378, 1439, 1552, 2078, 2593, 2771, 3601, 3654, 4269, 5640, 5790, 
                5791, 6168, 6418, 6591, 7651, 7714, 7827, 7961, 8466, 8515, 8531, 8911, 9844, 
                11878, 10598, 11841, 11957, 11958, 11959, 12027, 12055, 12062, 12064, 12069]
# >> solution: manual cleaning and calculating

zero_dim = [3041, 8617, 10122, 10123, 10124]
# >> solution: remove the zero dimension, which has the same effect as setting the zero dim = 1

missing_size = [7669, 11945, 12300]
# >> solution: add the missing size on the medium field to the size field

pd.set_option('display.max_colwidth', 100)
saffronart_df.loc[multi_pieces+zero_dim+missing_size, ['title', 'artist', 'size', 'medium']]

,title,artist,size,medium
55,a) Sukai b) Pancho's Sister c) Pancho,navjot,a) height : 14 x 11.5 in,fiberglass and paint
990,"A) UNTITLED B) MONTH OF BAISAKHA, BARAMASA (KANGRA MINIATURE)",deepak bhandari,29.2 x 22.8 cm,b) 11.5 x 9 in
1254,Untitled,sakti burman,b) 15 x 12.5 in,a) 12 x 9.5 in (30.4 x 24.1 cm)
1378,"UNTITLED, a) 1992 b) 2014",sakti burman,5 x 3.5 in,b) ink on paper pasted on mountboard
1439,"UNTITLED, a) 2007 b) 2011",arpana caur,10.5 x 14.5 in,b) mixed media on handmade paper
1552,We Who Live by Myth,phaneendra nath chaturvedi,44.25 x 30 in,this work comprises of four panels
2078,"A) ON DEMAND, B) JUST MARRIED, 2005",sunoj d.,34.5 x 24.5 in,a) silk screen on imposed print and sand blasted glass
2593,An Old (Oslo) Story,anita dube,two panels measuring 13.5 x 8.5 in,laminated photo-copies and fabric transfer mounted on board
2771,a) Swan Song Callb) Sugar and Milk,chitra ganesh,29.5 x 43.5 in,b) digital print on paper
3601,Untitled,somnath hore,14 x 9.5 in,b) charcoal and wash on paper


In [8]:
# add the missing size on the medium field to the size field:
saffronart_df.at[[7669, 11945], 'size'] =+ saffronart_df.loc[[7669, 11945], 'medium']

# for record 12300th, just add the dimension in inch:
saffronart_df.at[12300, 'size'] = saffronart_df.at[12300, 'size'] + ' 15.5 in' 

saffronart_df.loc[missing_size, ['size', 'medium']]

,size,medium
7669,â â 146 cm x 105.4 cm,â â 146 cm x 105.4 cm
11945,107.9 x 160 cm (each),107.9 x 160 cm (each)
12300,28 x 7 in 15.5 in,height: 15.5 in (39.3 cm)


In [9]:
def get_area_vol(x):
    dimensions = re.findall('\d+\.\d+|\d+', x)  
    dimensions = list(map(float, dimensions)) # turn all dimensions into float type
    dimensions = [d for d in dimensions if d > 0] # remove invalid dimension that's equal 0
                       
    if 'cm' in x:
        dimensions = np.array(dimensions)/2.54 # convert to inch
    
    return np.prod(dimensions) # multiply them altogether

In [10]:
saffronart_df['area_or_vol'] = saffronart_df['size'].apply(lambda x: get_area_vol(x))
saffronart_df['area_or_vol']

0       5112.25
1       1430.00
2       2209.00
3       1468.50
4       1600.00
          ...  
12416    441.00
12417   5428.00
12418    141.75
12419   1259.25
12420    864.00
Name: area_or_vol, Length: 12421, dtype: float64

In [11]:
saffronart_df.loc[multi_pieces, ['title', 'artist', 'auction_date', 'size', 'medium', 'area_or_vol']]

,title,artist,auction_date,size,medium,area_or_vol
55,a) Sukai b) Pancho's Sister c) Pancho,navjot,2007-09-01,a) height : 14 x 11.5 in,fiberglass and paint,161.00
990,"A) UNTITLED B) MONTH OF BAISAKHA, BARAMASA (KANGRA MINIATURE)",deepak bhandari,2015-11-01,29.2 x 22.8 cm,b) 11.5 x 9 in,103.19
1254,Untitled,sakti burman,2015-02-12,b) 15 x 12.5 in,a) 12 x 9.5 in (30.4 x 24.1 cm),187.50
1378,"UNTITLED, a) 1992 b) 2014",sakti burman,2015-07-01,5 x 3.5 in,b) ink on paper pasted on mountboard,17.50
1439,"UNTITLED, a) 2007 b) 2011",arpana caur,2015-01-01,10.5 x 14.5 in,b) mixed media on handmade paper,152.25
1552,We Who Live by Myth,phaneendra nath chaturvedi,2017-06-01,44.25 x 30 in,this work comprises of four panels,1327.50
2078,"A) ON DEMAND, B) JUST MARRIED, 2005",sunoj d.,2015-10-01,34.5 x 24.5 in,a) silk screen on imposed print and sand blasted glass,845.25
2593,An Old (Oslo) Story,anita dube,2013-09-01,two panels measuring 13.5 x 8.5 in,laminated photo-copies and fabric transfer mounted on board,114.75
2771,a) Swan Song Callb) Sugar and Milk,chitra ganesh,2011-02-01,29.5 x 43.5 in,b) digital print on paper,1283.25
3601,Untitled,somnath hore,2015-06-01,14 x 9.5 in,b) charcoal and wash on paper,133.00


In [12]:
# manually fix the area and volume of lots with multiple pieces:

# these details are taken from saffronart.com or storyltd.com
# some aren't easily retrieved by Google search so I have to make assumptions about them
# for example, assume that two pieces have the same size

saffronart_df.at[55, 'area_or_vol'] = (14*7.5*11.5) + (14*7*11) + (14*9*10)
saffronart_df.at[990, 'area_or_vol'] = (11*8.5) + (11.5*9)
saffronart_df.at[1254, 'area_or_vol'] = (15*12.5) + (12*9.5)
saffronart_df.at[1378, 'area_or_vol'] = 5*3.5*2
saffronart_df.at[1439, 'area_or_vol'] = (10.5*14.5) + (7.5*11)
saffronart_df.at[1552, 'area_or_vol'] = 44.25*30*4
saffronart_df.at[2078, 'area_or_vol'] = (34.5*24.5) + (30.5*41) 
saffronart_df.at[2593, 'area_or_vol'] = 13.5*8.5*2 + 10*15.5*2 + 11*11.5*2 + 10*13.5*2 + 11.5*31.5*2 + 9.5*13.5*2 + 11.5*15.5*4
saffronart_df.at[2771, 'area_or_vol'] = 29.5*43.5 + 25.5*41.5
saffronart_df.at[3601, 'area_or_vol'] = 14*9.5*5
saffronart_df.at[5640, 'area_or_vol'] = 6.45*4.44 + 6.41*4.33
saffronart_df.at[5790, 'area_or_vol'] = 10*8*2
saffronart_df.at[5791, 'area_or_vol'] = 7*10.5*2
saffronart_df.at[6168, 'area_or_vol'] = 26.5*56.5 + 33.5*24.5
saffronart_df.at[6418, 'area_or_vol'] = 46.5*46.5*2
saffronart_df.at[7651, 'area_or_vol'] = 3.5*5.5 + 5.5*3.5*2
saffronart_df.at[7714, 'area_or_vol'] = 7.5*7.25*15.5 + 7*11.75*7
saffronart_df.at[7827, 'area_or_vol'] = 24*18*2
saffronart_df.at[7961, 'area_or_vol'] = 11*7*2 + 11*7.5*4
saffronart_df.at[8466, 'area_or_vol'] = 4.5*4.5 + 10.5*8.5 
saffronart_df.at[8515, 'area_or_vol'] = 11*8.5 + 10*8*2
saffronart_df.at[8531, 'area_or_vol'] = 7*6.5 + 8*7 + 8*7.5 
saffronart_df.at[9844, 'area_or_vol'] = 17.5*17.5 + 17*17
saffronart_df.at[11878, 'area_or_vol'] = 40*17.5*17.5 + 12*17*17
saffronart_df.at[10598, 'area_or_vol'] = 32*20.5 + 30*11 + 24*37.5 
saffronart_df.at[11841, 'area_or_vol'] = 24*12 + 12*36
saffronart_df.at[11957, 'area_or_vol'] = 24.75*18.25 + 21*17.25
saffronart_df.at[11958, 'area_or_vol'] = 8*5*2 + 9*5 + 7.5*5 + 8.5*5
saffronart_df.at[11959, 'area_or_vol'] = 12*9 + 28.5*22.5  
saffronart_df.at[12027, 'area_or_vol'] = 30*22*2 + 30.5*22.5
saffronart_df.at[12055, 'area_or_vol'] = 25*10*2
saffronart_df.at[12062, 'area_or_vol'] = 21*14 + 13.5*5*4
saffronart_df.at[12064, 'area_or_vol'] = 8*2*2 + 9.5*3 + 10*3
saffronart_df.at[12069, 'area_or_vol'] = 18*11 + 14*5*4

In [13]:
# verify the minimum and maximum values of area_or_vol:

print('MIN AREA/VOLUME:\n', saffronart_df.iloc[saffronart_df.area_or_vol.idxmin()], sep='')
print()
print('MAX AREA/VOLUME:\n', saffronart_df.iloc[saffronart_df.area_or_vol.idxmax()], sep='')

MIN AREA/VOLUME:
artist                           sojwal samant
birth_date                 1972-01-01 00:00:00
birth_place                             Baroda
death_date                                 NaT
artist_age                               47.00
title                                 Untitled
winning_bid                             850.00
low_est                                1045.00
high_est                               1250.00
auction_date               2011-11-01 00:00:00
category                             sculpture
style                               figurative
size                              0.2 x 5.5 in
medium           papier mache and textile dye 
area_or_vol                               1.10
Name: 9721, dtype: object

MAX AREA/VOLUME:
artist              sudhanshu sutar
birth_date      1969-01-01 00:00:00
birth_place        Kalikapur Orissa
death_date                      NaT
artist_age                    50.00
title                      Untitled
winning_bid       

---
# Compute artist's age (on auction date)

In [14]:
# get the artist's age on the date of the auction:

saffronart_df['auction_dt_age'] = saffronart_df['auction_date'].sub(saffronart_df['birth_date'], axis=0)
saffronart_df.auction_dt_age

0       23162 days
1       21124 days
2       21032 days
3       20851 days
4       20851 days
           ...    
12416   18852 days
12417   36877 days
12418   27271 days
12419   21646 days
12420   25080 days
Name: auction_dt_age, Length: 12421, dtype: timedelta64[ns]

In [15]:
# divide the number of days by 365 to get the age:

saffronart_df['auction_dt_age'] = saffronart_df['auction_dt_age'].dt.days//365
saffronart_df.auction_dt_age

0        63.00
1        57.00
2        57.00
3        57.00
4        57.00
         ...  
12416    51.00
12417   101.00
12418    74.00
12419    59.00
12420    68.00
Name: auction_dt_age, Length: 12421, dtype: float64

In [16]:
# if the artist is no longer living at the time of the auction, auction_dt_age = artist_age

saffronart_df.loc[saffronart_df.auction_dt_age > saffronart_df.artist_age, 'auction_dt_age']=saffronart_df.artist_age

In [17]:
# verify the minimum and maximum values of auction_dt_age:

print('YOUNGEST ARTIST:\n', saffronart_df.iloc[saffronart_df.auction_dt_age.idxmin()], sep='')
print()
print('OLDEST ARTIST:\n', saffronart_df.iloc[saffronart_df.auction_dt_age.idxmax()], sep='')

YOUNGEST ARTIST:
artist                girish dahiwale
birth_date        1974-01-01 00:00:00
birth_place                       NaN
death_date        1998-01-01 00:00:00
artist_age                      24.00
title                        Untitled
winning_bid                   9178.00
low_est                       2900.00
high_est                      4350.00
auction_date      2018-12-01 00:00:00
category                     painting
style                      figurative
size                 95.75 x 113.5 in
medium              acrylic on cloth 
area_or_vol                  10867.62
auction_dt_age                  24.00
Name: 2083, dtype: object

OLDEST ARTIST:
artist                     b c sanyal
birth_date        1904-04-22 00:00:00
birth_place           Dibrugarh Assam
death_date        2003-01-01 00:00:00
artist_age                      99.00
title                        Untitled
winning_bid                       NaN
low_est                      11595.00
high_est                     

In [18]:
# export the updated dataframe to a CSV for later use: 
saffronart_df.to_csv("updated_saffronart_dataset.csv", index = False)